In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import json
from multithread_processing.base_job import BaseJob

from databases.mongodb import MongoDB
from databases.mongodb_entity import MongoDBEntity

db = MongoDB()
entity_db = MongoDBEntity()

In [2]:
class CheckExistWallets(BaseJob):
    def __init__(self, wallets_list: list, check_func, filename: str,
                 max_workers=8, batch_size=1000):
        self.check_func = check_func
        self.filename = filename

        self.exist_wallets = list()

        super().__init__(work_iterable=wallets_list,
                         max_workers=max_workers,
                         batch_size=batch_size)

    def _execute_batch(self, works):
        for address in works:
            datum = self.check_func(address)
            # datum = db.get_lending_wallet(address)
            if datum:
                self.exist_wallets.append(datum)

    def _end(self):
        super()._end()
        with open(self.filename, 'w') as f:
            json.dump(self.exist_wallets, f, indent=2)

0x38

In [3]:
data = pd.read_csv('.data/0x38_wallets_pairs.csv')
data.head()

,Unnamed: 0,x,y
0,0,0x7826d1fd382d877a2f202409c556fe21bc0432e9,0xef59258c9dd77d650b564e08591a97d86568c46d
1,1,0x6376f6e4fec4ec5aba9b8bd2d6d85c610eee0854,0x08d081f9e8af4f7735387a8fc727c13e22d3d44b
2,2,0xb1ff25df6ecdd5f94b0ba7b3afdc8a44758cad9f,0x7f05aad364d5b2dc641058128362377a193564cf
3,3,0x125abbecbf3172b5a57133893f73ca4a53504e1b,0x0d31d2e6cc8784302a0fa71f50f486ccebb85dbd
4,4,0x25324601c880f232c0559b95ce459efbe82a9575,0x18d05ff1e903b945f87c61c521ded1994555b2f0


In [6]:
job = CheckExistWallets(max_workers=16,
                        batch_size=1000,
                        wallets_list=list(data['y']),
                        check_func=db.get_lending_wallet,
                        filename='.data/exist_lending_wallets_y.json')

job.run()

[08-09-2023 17:15:23 +07] [INFO] [BatchWorkExecutor] - Started work. Items to process: 37471.
[08-09-2023 17:16:45 +07] [INFO] [BatchWorkExecutor] - 4000 items processed. Progress is 10%.
[08-09-2023 17:16:45 +07] [INFO] [BatchWorkExecutor] - 8000 items processed. Progress is 21%.
[08-09-2023 17:16:46 +07] [INFO] [BatchWorkExecutor] - 12000 items processed. Progress is 32%.
[08-09-2023 17:16:47 +07] [INFO] [BatchWorkExecutor] - 15000 items processed. Progress is 40%.
[08-09-2023 17:17:59 +07] [INFO] [BatchWorkExecutor] - 19000 items processed. Progress is 50%.
[08-09-2023 17:18:00 +07] [INFO] [BatchWorkExecutor] - 23000 items processed. Progress is 61%.
[08-09-2023 17:18:00 +07] [INFO] [BatchWorkExecutor] - 27000 items processed. Progress is 72%.
[08-09-2023 17:18:02 +07] [INFO] [BatchWorkExecutor] - 30000 items processed. Progress is 80%.
[08-09-2023 17:19:13 +07] [INFO] [BatchWorkExecutor] - 34471 items processed. Progress is 91%.
[08-09-2023 17:19:14 +07] [INFO] [BatchWorkExecutor] 

In [12]:
with open('.data/exist_lending_wallets.json', 'r') as f:
    exist_lending_wallets = json.load(f)
print("Number of lending wallet:", len(exist_lending_wallets))

Number of lending wallet: 689


In [13]:
job = CheckExistWallets(max_workers=16,
                        batch_size=1000,
                        wallets_list=list(data['x']),
                        check_func=db.get_lp_deployer_wallet,
                        filename='.data/exist_deployer_wallets.json')
job.run()

[08-09-2023 17:20:54 +07] [INFO] [BatchWorkExecutor] - Started work. Items to process: 37471.
[08-09-2023 17:22:07 +07] [INFO] [BatchWorkExecutor] - 4000 items processed. Progress is 10%.
[08-09-2023 17:22:07 +07] [INFO] [BatchWorkExecutor] - 8000 items processed. Progress is 21%.
[08-09-2023 17:22:08 +07] [INFO] [BatchWorkExecutor] - 12000 items processed. Progress is 32%.
[08-09-2023 17:22:08 +07] [INFO] [BatchWorkExecutor] - 15000 items processed. Progress is 40%.


In [ ]:
with open('.data/exist_deployer_wallets.json', 'r') as f:
    exist_lending_wallets = json.load(f)
print("Number of deployer wallet:", len(exist_lending_wallets))

In [ ]:
job = CheckExistWallets(max_workers=16,
                        batch_size=1000,
                        wallets_list=list(data['x']),
                        check_func=db.get_lp_trader_wallet,
                        filename='.data/exist_trader_wallets.json')

job.run()

In [ ]:
with open('.data/exist_trader_wallets.json', 'r') as f:
    exist_lending_wallets = json.load(f)
print("Number of trader wallet:", len(exist_lending_wallets))